In [3]:
import os
import sys
import math
import numpy as np
import pandas as pd
import pickle as pkl
import tifffile as tif
from keras.layers import Dense
from keras.layers import Conv2D
from multiprocessing import Pool
from keras.utils import Sequence
from keras.layers import Flatten
from keras.layers import MaxPool2D
from keras.layers import Concatenate
from keras.layers import Merge
from keras.models import Sequential
os.chdir("../../")

Using TensorFlow backend.


In [4]:
class ImageDataGenerator(Sequence):
    
    def __init__(self, x_metadata, y_metadata, batch_size, crop_size):
        self.x = x_metadata
        self.y = y_metadata
        self.batch_size = batch_size
        self.cp = crop_size
    
    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))
    
    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]
        
        return [np.array([np.transpose(np.array(tif.imread(file_name), dtype=int)/255.0,
                                     (1,2,0))[self.cp:-self.cp,self.cp:-self.cp,:]
                        for file_name in batch_x]), np.array([np.transpose(np.array(tif.imread(file_name), dtype=int)/255.0,
                                     (1,2,0))[self.cp:-self.cp,self.cp:-self.cp,:]
                        for file_name in batch_x])], np.array(batch_y)         

class CNN_Model:
    
    def __init__(self, directory):
        
        self.path = directory
        self.classes = sorted(list(os.listdir(self.path+"/train")))
        self.train_metadata_x = []
        self.train_metadata_y = []
        self.test_metadata_x = []
        self.test_metadata_y = []
        
        for cls in self.classes:
            for im in os.listdir(self.path+"/train/"+cls):
                self.train_metadata_x.append(self.path+"/train/"+cls+"/"+im)
        
        for cls in self.classes:
            for im in os.listdir(self.path+"/test/"+cls):
                self.test_metadata_x.append(self.path+"/test/"+cls+"/"+im)
        
        np.random.shuffle(self.train_metadata_x)
        np.random.shuffle(self.test_metadata_x)
        
        for p in self.train_metadata_x:
            y = np.zeros(10, dtype=int)
            y[self.classes.index(p.split("/")[3])] = 1
            self.train_metadata_y.append(y)
            
        for p in self.test_metadata_x:
            y = np.zeros(10, dtype=int)
            y[self.classes.index(p.split("/")[3])] = 1
            self.test_metadata_y.append(y)
    
    def model_create(self):
        
        # Branch 1
        branch1 = Sequential()
        # Step 1 - Convolution
        branch1.add(Conv2D(filters=96, kernel_size=(2, 2), input_shape = (32,32,33), activation = 'relu'))
        branch1.add(Conv2D(filters=288, kernel_size=(2, 2), activation = 'relu'))
        branch1.add(Conv2D(filters=576, kernel_size=(2, 2), activation = 'relu'))
        branch1.add(Conv2D(filters=64, kernel_size=(2, 2), activation = 'relu'))
        branch1.add(MaxPool2D(pool_size = (2, 2)))
        branch1.summary()
        
        # Branch 2
        branch2 = Sequential()
        # Step 1 - Convolution
        branch2.add(Conv2D(filters=96, kernel_size=(2, 2), input_shape = (32,32,33), activation = 'relu'))
        branch2.add(Conv2D(filters=288, kernel_size=(2, 2), activation = 'relu'))
        branch2.add(Conv2D(filters=576, kernel_size=(2, 2), activation = 'relu'))
        branch2.add(Conv2D(filters=64, kernel_size=(2, 2), activation = 'relu'))
        branch2.add(MaxPool2D(pool_size = (2, 2)))
        branch2.summary()
        
        # Step 3 - Flattening
        classifier = Sequential()
        classifier.add(Merge([branch1, branch2], mode='concat', concat_axis=3))
        classifier.add(Flatten())
        # Step 4 - Full connection
        classifier.add(Dense(256, activation = 'relu'))
        classifier.add(Dense(128, activation = 'tanh'))
        classifier.add(Dense(10, activation = 'sigmoid'))
        # Compiling the CNN
        classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
        classifier.summary()
        return classifier
    
    def fit_generator(self, num_epochs=10, batch_size=32, crop_size=16):        
        
        classifier = self.model_create()
        train_data = ImageDataGenerator(self.train_metadata_x, self.train_metadata_y, batch_size, crop_size)
        test_data = ImageDataGenerator(self.test_metadata_x, self.test_metadata_y, batch_size, crop_size)
        history = classifier.fit_generator(train_data, epochs=num_epochs, use_multiprocessing=True,shuffle=True)
        classifier.evaluate_generator(test_data, use_multiprocessing=True)

In [5]:
ob = CNN_Model(directory="Data/Class wise Data")

In [6]:
ob.fit_generator(num_epochs=15, batch_size=100, crop_size=16)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 31, 31, 96)        12768     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 30, 30, 288)       110880    
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 29, 29, 576)       664128    
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 28, 28, 64)        147520    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 64)        0         
Total params: 935,296
Trainable params: 935,296
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:78: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.


Epoch 1/15


ResourceExhaustedError: OOM when allocating tensor with shape[100,576,29,29] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[Node: training/Adam/gradients/conv2d_8/convolution_grad/Conv2DBackpropInput = Conv2DBackpropInput[T=DT_FLOAT, _class=["loc:@conv2d_8/convolution"], data_format="NCHW", dilations=[1, 1, 1, 1], padding="VALID", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](training/Adam/gradients/conv2d_8/convolution_grad/ShapeN, conv2d_8/kernel/read, training/Adam/gradients/conv2d_8/Relu_grad/ReluGrad)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


Caused by op 'training/Adam/gradients/conv2d_8/convolution_grad/Conv2DBackpropInput', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", 